# Batchnorm

### I. 思路
#### 背景和想解决的问题
- DNN训练过程中有两个典型问题导致收敛难。一个是covariate shift，另一个是梯度消失。
  - covariate shift：迭代过程中每层input分布变化导致难收敛
- 本文提出了一种方法：fix每层的means和variances。
- 目的：是处理covariate shift
- 作用：
  1. 处理covariate shift
  2. 同时也减少了梯度消失。因为可以减少梯度计算对w的scale的依赖。
  3. 起到了regularization的作用，减少了模型中使用dropout的需求
  4. 让模型可以使用saturating non-linearities，因为它能减轻网络计算大量出现saturated neurals的情况
  5. 让模型可以使用更大的learning rate，从而更快收敛。
  - <font color=blue>前几个都在BGD的notes中有说明，这里只分析最后一个作用产生的过程。</font>

#### 为什么可以用更大的learning rate
1. 在NN中，如果learning rate设置过大会有什么后果？
   - 在凸优化的分析中，典型结果是导致w的迭代路径出现fluctuate。
   - 从SGD的迭代过程来看，learning rate太大会让w在迭代后的scale较大，如果non-linearity是sigmoid或者tanh，就很容易saturate。
2. Batchnorm可以让迭代过程对w的scale不那么敏感，因为它控制了$x@w$的分布。
   - <font color=blue>假如w的scale扩大a倍，BN会让网络不受影响，因为$BN(a*x@w)=BN(x@w)$</font>
   - 证明：
$$\begin{align}
BN(a*s) & = \gamma*\frac{a*s - mean(a*s)}{\sqrt{var(a*s)}}+\beta \\ 
& = \gamma*\frac{a*s - a*mean(s)}{\sqrt{a^2var(s)}}+\beta \\
& = \gamma*\frac{s - mean(s)}{\sqrt{var(s)}}+\beta \\
& = BN(s)
\end{align}$$

#### 思路
1. <font color=norange>whitened input可以提高NN的训练效率，但是运算量大，并且会降低NN的表达力。</font>whiten input把数据改成均值为0，方差为1，这会导致non-linearity activation function的表达能力降低。所以考虑把whiten操作做一些调整。
   - 运算量大是因为要计算Hessian matrix
   - 表达能力降低的例子：比如用sigmoid或者tanh，如果input在（0， 1）范围内的话，这两个函数在这个input范围内的曲线很接近直线。所以non-linearity的能力被削弱了。这会导致NN的表达力降低。
2. 为了降低运算量，做简化：不考虑input中feature间的关系，直接norm单个feature。
   - $$\hat x = \frac{x - mean(x)}{var(x)}$$这里x是input中的单个元素，而不是整个input。
3. <font color=green>**为了让修改后的模型不损害NN的表达力，每层加入的transformation就应该能够表达identity transform。**</font>
   - 本文提出的方法是：给每个$\hat x$加两个参数gamma和beta，做为它的均值和方差。
   - $$\hat x = \frac{x - mean(x)}{\sqrt{var(x)+\epsilon}}, \ y = \gamma \hat x + \beta \\
   当 \gamma =  var(x), \beta = mean(x)时, y = x，\therefore可以表达identity function$$
   - 这些参数和模型中的其他参数$W,b$一起learn
4. Batchnorm加入network的位置：<font color=green>**在affine层后面，non-linearity层前面。**</font>
   - $x$是上一层non-linearity的输出，在整个training过程中，分布都在变化。有论文分析表明，$Wx比x$的分布更稳定，更接近Gaussian。
   - 加在affine后面的另一个好处是，同时把$Wx$的结果都norm了，这样non-linearity function拿到的输入的分布更好。可以减少non-lieanrity发生saturate的程度。

### II. 算法
#### 伪码
 - <img src="pics/batchnormAlg1.png" alt="alt text" width="320"/> 
 - <img src="pics/batchnormAlg2.png" alt="alt text" width="320"/>
 - **注：**
   - <font color=red>x是单个元素，是$x^{k}$的简写。</font>
   - $x^{k}$指一个input vector在第k个维度上的取值,$k\in \{1, ..., k\}$是dimension的index。
   - $x^{k}_i$指第i个样本对应的vector在第k个维度上的取值


#### 计算细节
1. 计算mean和var的方式：
   - NN训练一般都用SGD，mean和var的计算很难用整个training set，所以简化用minibatch数据来计算mean和var。
   - 在inference阶段，不再用minibatch，而是用整个training set的数据来计算。这里要注意var要用sample variance。并且$\hat x$不再额外加$\epsilon$。$$
   var_{sample}(x) = \frac{N}{N-1}var(x) \\
   \hat x = \frac{x-mean(x)}{\sqrt{var_{sample}(x)}}
   $$
2. 在CNN中，为了<font color=norange>让norm过程符合convolutional property，所以让同一个feature map上的所有元素以相同的方式被normalize。</font>
   - <font color=deeppink>**feature map是就是kernel map**</font>，是kernel对input做convolution得到的结果。比如一张image就会对应一个map。
     - 一个image样本维度记为$3\times H_{in}\times W_{in}$，一个kernel处理1个image样本会得到一个2D矩阵，这就是这个样本的feature map，维度是$H_{out}\times W_{out}$。
     - 如果是一个样本数量为m的minibatch，input的维度是$M \times 3\times H_{in}\times W_{in}$，一个kernel处理1个image样本会得到M个feature map。每个维度是$H_{out}\times W_{out}$，合计是$M \times H_{out}\times W_{out}$。
   - **建议的处理方式**：
     - 每个kernel处理生成的M个feature maps计算1对mean scalar和variance scalar，也对应有一个gamma和beta scalar pair。

### II. 效果分析：以MLP为例

1. **不同batchsize下，用和不使用bn的accuracy对比图**
 - **epoch=50，baseline是no batchnorm，batch=5**
<img src="pics/batchnorm1.png" alt="alt text" width="560"/>
 - **epoch=50，baseline是no batchnorm，batch=10(图上标记有误)**
<img src="pics/batchnorm2.png" alt="alt text" width="560"/>
 - **epoch=50，baseline是no batchnorm，batch=50(图上标记有误)**
<img src="pics/batchnorm3.png" alt="alt text" width="560"/>


2. **batchsize对batchnorm效果的影响和原因分析**
    1. **batchsize较小的时候，batchnorm不仅不提升，反而降低training accuracy**。**batchsize越大，training accuracy越高**。随着batchsize打到一定数量后，做batchnorm的training accuracy也能达到不做batchnorm的模型水平。
       - **分析：**其他条件相同的情况下，做batchnorm相当于增加了Regularization，使模型更不容易发生overfit，而且，batchsize越小，由于抽样带来的noise越大，Regularization strength也越大。所以当batchsize仅仅为5的时候，不用batchnorm会发生overfit，而用了batchnorm则不会。随着batchsize增加，Regularization strength下降，training accuracy就会增加。
    2. 做了batchnorm的模型中，batchsize越大，**validation accuracy和training accuracy提升的速度更快**。***[这是batchnorm的最大价值点：加速优化过程]***
       - **分析：**batchsize越大，batch mean和batch variance值的波动更小，并且更能体现数据本身的真实的统计特征。This leads to more accurate normalization.从而使batchnorm的效果更好。
    3. 做了batchnorm的模型中，当训练达到一定epoch之后，**不同的batchsize对应的best validation accuracy区别不大**。
       - **分析：**当迭代epoch数量足够多了之后，即使batchsize不同，但running average of mean and variance的值趋于一致，从test阶段的计算规则可知，这会使得validation的结果和training不同，后者的accuracy会趋于一致。

## 3. 使用batchnorm遇到的问题

1. 由于batchnorm的效果受batchsize的影响较大。导致复杂的网络中，如果batchsize受硬件限制而很小的话，batchnorm的效果就很有限。比如online learning，此时batchsize=1。
2. 无法用到RNN上。因为句子长度不同会导致在batch里面计算feature的均值时，其波动特别大。因为很多feature都是sparse的，在一个batch里面甚至都不会出现，feature mean的波动就会很大。此外，inference阶段如果有很长的句子，那么training阶段获得的全局均值和方差对它很可能不使用

为了解决这个问题，就有了layernorm和groupnorm等其他方法。